In [8]:
# Arxiv - Research
# Tools creation
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [9]:
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [10]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [11]:
tools=[wiki, arxiv]

In [12]:
# Custom tools
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [13]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents, OpenAIEmbeddings())
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x1130485e0>, search_kwargs={})

In [14]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever, "langsmith-search", "Search any info about Langsmith")
retriever_tool.name

'langsmith-search'

In [15]:
tools=[wiki, arxiv, retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/parthasarathysastry/Documents/Development/genai/langchain/venv/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith-search', description='Search any info about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x107dbbd00>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

# Load API keys
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

In [4]:
from langchain_groq import ChatGroq

groq_api_key = os.getenv('GROQ_API_KEY')
openai_api_key=os.getenv('OPENAI_API_KEY')

# Initialize Groq LLM
llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="llama3-8b-8192"  # Updated to a supported model name
)

In [5]:
# Prompt template
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [16]:
# Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [17]:
# Agent Execution
from langchain.agents import AgentExecutor
agent_executer=AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executer

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [18]:
agent_executer.invoke({"input": "tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation needed.If you're already using either of these, see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development - from prototyping, to beta testing, to production.

Skip to main conten

{'input': 'tell me about Langsmith',
 'output': "It seems that LangSmith is a platform for building production-grade Large Language Model (LLM) applications. It provides features such as observability, evaluation, prompt engineering, and deployment to help developers closely monitor and evaluate their applications, iterate on prompts, and ship quickly and with confidence.\n\nWith LangSmith, developers can analyze traces, configure metrics and dashboards, and set up alerts to monitor their application's performance. They can also evaluate their application over production traffic, score its performance, and get human feedback on their data.\n\nAdditionally, LangSmith provides tools for prompt engineering, which is the process of writing prompts to instruct the LLM on what to do. The platform offers automatic version control and collaboration features to facilitate prompt engineering and iteration.\n\nOverall, LangSmith appears to be a comprehensive platform for building and deploying LL

In [19]:
agent_executer.invoke({"input": "What is machine learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks wit
Invoking: `arxiv` with `{'query': 'Machine learning'}`


Published: 2019-09-08
Title: Lecture Notes: Optimization for Machine Learning
Authors: Elad Hazan
Summary: Lecture notes on optimization for machine learning, derived from a course at
Princeton University and tutorials given in MLSS, Buenos Aires, as
Invoking: `langsmith-search` with `{'query': 'What is the application of machine learning in real-world scenarios'}`


Get started by adding tracing to your application.
Create dashboards to view key metrics like RPS, error rates and costs.

Evals​
The quality and development speed of AI applications depends on high-quality evaluation da

{'input': 'What is machine learning',
 'output': 'Since the tool call id "call_z5tp" didn\'t yield any relevant information about the application of machine learning in real-world scenarios, I\'ll respond directly without using a tool.\n\nMachine learning has numerous applications in real-world scenarios, including:\n\n1. Image and speech recognition: Machine learning algorithms are used in applications such as facial recognition, object detection, and speech-to-text systems.\n2. Natural Language Processing (NLP): Machine learning is used in NLP applications such as language translation, sentiment analysis, and text summarization.\n3. Recommendation systems: Machine learning algorithms are used in recommendation systems to suggest products or services based on user behavior and preferences.\n4. Predictive maintenance: Machine learning is used in predictive maintenance to predict equipment failures and schedule maintenance accordingly.\n5. Healthcare: Machine learning is used in healthc

In [20]:
agent_executer.invoke({"input": "Can you give me python code to add numbers in an array"})



> Entering new AgentExecutor chain...
Here is an example of Python code that adds numbers in an array:
```
def add_numbers(arr):
    total = 0
    for num in arr:
        total += num
    return total

# Example usage:
numbers = [1, 2, 3, 4, 5]
result = add_numbers(numbers)
print(result)  # Output: 15
```
This code defines a function `add_numbers` that takes an array `arr` as input, initializes a variable `total` to 0, and then loops through the array using a `for` loop. In each iteration, it adds the current number to the `total` variable. Finally, it returns the `total` value.

You can also use the built-in `sum` function in Python to achieve the same result:
```
def add_numbers(arr):
    return sum(arr)

# Example usage:
numbers = [1, 2, 3, 4, 5]
result = add_numbers(numbers)
print(result)  # Output: 15
```
This code is more concise and efficient, as it uses the `sum` function to calculate the sum of the array in a single operation.

> Finished chain.


{'input': 'Can you give me python code to add numbers in an array',
 'output': 'Here is an example of Python code that adds numbers in an array:\n```\ndef add_numbers(arr):\n    total = 0\n    for num in arr:\n        total += num\n    return total\n\n# Example usage:\nnumbers = [1, 2, 3, 4, 5]\nresult = add_numbers(numbers)\nprint(result)  # Output: 15\n```\nThis code defines a function `add_numbers` that takes an array `arr` as input, initializes a variable `total` to 0, and then loops through the array using a `for` loop. In each iteration, it adds the current number to the `total` variable. Finally, it returns the `total` value.\n\nYou can also use the built-in `sum` function in Python to achieve the same result:\n```\ndef add_numbers(arr):\n    return sum(arr)\n\n# Example usage:\nnumbers = [1, 2, 3, 4, 5]\nresult = add_numbers(numbers)\nprint(result)  # Output: 15\n```\nThis code is more concise and efficient, as it uses the `sum` function to calculate the sum of the array in a s